In [52]:
import pandas as pd
! pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\modik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [53]:
df = pd.read_csv("spotify_millsongdata.csv")

In [54]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [55]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [56]:
df.shape

(57650, 4)

In [57]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [58]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [59]:
df.head(10)

,artist,song,text
0,Elton John,High Flying Bird,You wore a little cross of gold around your ne...
1,James Taylor,T-Bone,"Me and T-Bone on the road to town, \r\nIt's l..."
2,Pitbull,We Run The Night,Havannah Brown \r\nRed One \r\nMr.Worldwide ...
3,W.A.S.P.,The Running Man,"You'll be so far out of control \r\nYou know,..."
4,Phish,Runaway Jim,"I had a dog, his name was Jim \r\n(Runaway, r..."
5,Conway Twitty,Hero For A Day,"Lady, you need a lover \r\nSomeone to take yo..."
6,Marillion,Beautiful,Everybody knows that we live in a world \r\nW...
7,Wishbone Ash,Streets Of Shame,"I was a long way from home, on the streets of ..."
8,Kelly Family,Babylon,"I'm the anger, yeah, that you got in your bone..."
9,Hank Williams Jr.,Big Twenty,"Move along big twenty, big twenty move along ..."


In [60]:
df['text'][0]

"You wore a little cross of gold around your neck  \r\nI saw it as you flew between my reason  \r\nLike a raven in the night time when you left  \r\nI wear a chain upon my wrist that bears no name  \r\nYou touched it and you wore it  \r\nAnd you kept it in your pillow all the same  \r\n  \r\nMy high-flying bird has flown from out my arms  \r\nI thought myself her keeper  \r\nShe thought I meant her harm  \r\nShe thought I was the archer  \r\nA weather man of words  \r\nBut I could never shoot down  \r\nMy high-flying bird  \r\n  \r\nThe white walls of your dressing room are stained in scarlet red  \r\nYou bled upon the cold stone like a young man  \r\nIn the foreign field of death  \r\nWouldn't it be wonderful is all I heard you say  \r\nYou never closed your eyes at night and learned to love daylight  \r\nInstead you moved away\r\n\r\n"

In [61]:
# df = df.sample(5000)

In [62]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [63]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [64]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


In [65]:

def token(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [66]:
df['text'].apply(lambda x: token(x))

0       you wore a littl cross of gold around your nec...
1       me and t-bone on the road to town , it 's like...
2       havannah brown red one mr.worldwid from miami ...
3       you 'll be so far out of control you know , th...
4       i had a dog , hi name wa jim ( runaway , runaw...
                              ...                        
4995    hey whi 'd you wake me from my nap ? i 'm not ...
4996    if you think that you 're strong wan na fight ...
4997    when you found me my whole world chang i 'm re...
4998    greet card have all been sent the christma rus...
4999    [ intro ] if the weapon is your love , i got m...
Name: text, Length: 5000, dtype: object

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [69]:
similarity[0]

array([1.        , 0.01376719, 0.036229  , ..., 0.01770274, 0.00278938,
       0.01084509])

In [70]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [71]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [73]:
recommendation('High Flying Bird')

['Thought Of You',
 'Day Is Done',
 'Freebird',
 'Scar Tissue',
 "Wouldn't Be So Bad",
 'My Best Friend',
 'High On You Mama',
 'Like A Bird',
 "I Can't Wait",
 'Once In A While',
 'Something Wonderful',
 'Electricity',
 'The Raven',
 'Shoot Love',
 "You're Young And You'll Forget",
 'Hate',
 'Lost Stars',
 'My Man',
 'Still Under The Weather',
 'I Thought About You']

In [74]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))